In [ ]:
import cv2
import numpy as np
from ipywidgets import *
import matplotlib.pyplot as plt
import math
from math import atan2, cos, sin, sqrt, pi
from sympy import Point, Polygon, Line, Segment

#custom libraries
from TrajectoryPlanner import Node, Planner
from Vision import Vision

In [ ]:
#tuning parameters
min_blue = np.array([70, 80, 0])
min_red = np.array([70, 80, 0])

max_blue = np.array([70, 80, 0])
max_red = np.array([70, 80, 0])

min_green = np.array([21, 0, 0])
max_green = np.array([80, 255, 216])


#variables
x = 0 #tymio initial position
y = 0 #tymio initial position
angle = 0 #tymio angle
radius = 0 #tymio radius

goal_x = 0 #goal position
goal_y = 0 #goal position

vertices = [] #vertices



"""
vision = Vision()
x_center,y_center, goal_detected = vision.get_goal_position(frame)
center, radius, angle, thymio_detected = vision.get_thymio_position(frame,(0,0),0)
print("Position: {0} , goal: {1},{2}".format(center,x_center,y_center))
vertices = vision.get_obstacle_position(frame,radius,min_green,max_green)[0]"""

In [ ]:
#initial computer vision

#output should be x,y,angle,goal_position,vertices

In [ ]:
#Planning
planner = Planner()
planner.setup(vertices,[x,y],[goal_x,goal_y])
planner.plot()

In [ ]:
camera_counter = 0
state_obstacle = False #determines if we are in "path following" or "obstacle avoidance" mode
state_current_path_segment = 0 #which part of the path we are currently following

#this loop should be run multiple times a second
while np.sqrt((x-goal_x)**2 + (y-goal_y)**2) <= goal_threshold:
    
    #position estimation
    if camera_counter >= 50:
        frame = capture_camera()
        x,y,angle = estimate_position_camera(x,y,angle,frame)
    else:
        x,y,angle = estimage_position_no_camera(x,y,angle)
        
    #updates the "state obstacle if there is an obstacle"
        
    #follow path if there is no obstacle
    if not obstacle:
        controller_trajectory(x,y,angle,path) #changes the speed of tymio
    else:
        controller_obstacle_avoid(x,y,angle,path)
        if not obstacle:
            path = replan()
            
    camera_counter += 1
        